In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-145.jpg
/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-14.jpg
/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-210.jpg
/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-43.jpg
/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-369.jpg
/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-624.jpg
/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-861.jpg
/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-914.jpg
/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-383.jpg
/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-515.jpg
/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-768.jpg
/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-555.jpg
/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-360.jpg
/kaggle/input/cleanedbrain/cleaned/Training/pituitary/pituitary-9.

In [1]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.layers import Input,Dropout, Dense,GlobalAveragePooling2D,Flatten
from keras.models import Sequential,Model
from keras.applications.resnet import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import itertools
import cv2
import os
import io

In [2]:
labels = ['glioma', 'meningioma','notumor', 'pituitary']
x_train = [] 
y_train  = []
x_test = []
y_test = []

image_size = 200


for label in labels:
    trainPath = os.path.join('/kaggle/input/cleanedbrain/cleaned/Training',label)
    for file in tqdm(os.listdir(trainPath)):
        image = cv2.imread(os.path.join(trainPath, file),0) 
        image = cv2.bilateralFilter(image, 2, 50, 50) 
        image = cv2.applyColorMap(image, cv2.COLORMAP_BONE) 
        image = cv2.resize(image, (image_size, image_size)) 
        x_train.append(image)
        y_train.append(labels.index(label))
    
    testPath = os.path.join('/kaggle/input/cleanedbrain/cleaned/Testing',label)
    for file in tqdm(os.listdir(testPath)):
        image = cv2.imread(os.path.join(testPath, file),0)
        image = cv2.bilateralFilter(image, 2, 50, 50)
        image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
        image = cv2.resize(image, (image_size, image_size))
        x_test.append(image)
        y_test.append(labels.index(label))


x_train = np.array(x_train) / 255.0 
x_test = np.array(x_test) / 255.0

print(x_train.shape)
print(x_test.shape)

100%|██████████| 93/93 [00:00<00:00, 114.17it/s]


(4352, 200, 200, 3)
(712, 200, 200, 3)


In [5]:
print(y_train)

[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


In [4]:
x_train, y_train = shuffle(x_train,y_train, random_state=2)
y_train = tf.keras.utils.to_categorical(y_train) 
y_test = tf.keras.utils.to_categorical(y_test)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=4) #Dividing the dataset into Training and Validation sets.

print(x_val.shape)

(871, 200, 200, 3)


In [11]:
gc.collect()

NameError: name 'gc' is not defined

In [6]:
datagen = ImageDataGenerator(
    rotation_range=10,                        
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True)
datagen.fit(x_train)

In [6]:
net = ResNet50(
    weights='imagenet', 
     include_top=False, 
     input_shape=(image_size,image_size,3))

2023-01-30 06:02:37.941964: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 06:02:37.942973: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 06:02:38.095121: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 06:02:38.096103: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 06:02:38.096928: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

94781440/94765736 [==============================] - 1s 0us/step


In [7]:
model = net.output
model = GlobalAveragePooling2D()(model)
model = Dropout(0.4)(model)
model = Dense(4, activation="softmax")(model)
model = Model(inputs= net.input, outputs= model)


adam = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss = 'categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 100, 100, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [8]:
ES = EarlyStopping(monitor = 'val_loss',min_delta = 0.001,patience = 5,mode = 'min',restore_best_weights = True,verbose = 1)
history = model.fit(datagen.flow(x_train, y_train, batch_size=20),validation_data = (x_val,y_val),epochs = 50,callbacks =[ES])

2023-01-30 06:02:46.902477: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2023-01-30 06:02:54.214698: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


175/175 [==============================] - 69s 302ms/step - loss: 0.2945 - accuracy: 0.8926 - val_loss: 2.9276 - val_accuracy: 0.3215
Epoch 2/50
175/175 [==============================] - 46s 260ms/step - loss: 0.1180 - accuracy: 0.9618 - val_loss: 2.3891 - val_accuracy: 0.1676
Epoch 3/50
175/175 [==============================] - 46s 259ms/step - loss: 0.0822 - accuracy: 0.9739 - val_loss: 2.3275 - val_accuracy: 0.3226
Epoch 4/50
175/175 [==============================] - 46s 262ms/step - loss: 0.0556 - accuracy: 0.9828 - val_loss: 3.0199 - val_accuracy: 0.4305
Epoch 5/50
175/175 [==============================] - 47s 267ms/step - loss: 0.0607 - accuracy: 0.9819 - val_loss: 1.7315 - val_accuracy: 0.6119
Epoch 6/50
175/175 [==============================] - 46s 261ms/step - loss: 0.0377 - accuracy: 0.9865 - val_loss: 0.5227 - val_accuracy: 0.8829
Epoch 7/50
175/175 [==============================] - 46s 262ms/step - loss: 0.0293 - accuracy: 0.9908 - val_loss: 0.1874 - val_accuracy: 0.9

In [9]:
loss,acc = model.evaluate(x_test,y_test)

23/23 [==============================] - 2s 107ms/step - loss: 0.1958 - accuracy: 0.9424


In [10]:
model.save('/kaggle/working/resmod.h5')

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [5]:
from tensorflow.keras.applications.vgg19 import VGG19
vgg=VGG19(include_top=False,
    weights='imagenet',
         input_shape=(image_size,image_size,3))

2023-01-30 16:53:13.166710: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 16:53:13.167741: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 16:53:13.366233: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 16:53:13.367056: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 16:53:13.368017: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [6]:
for layer in vgg.layers:
    layer.trainable=False

In [9]:
model=vgg.output
#model = GlobalAveragePooling2D()(model)
model=Flatten()(model)
#model=Dense(10000,activation='relu')(model)
#model=Dense(1000,activation='relu')(model)
#model = Dropout(0.4)(model)
model=Dense(4,activation='softmax')(model)
vgg = Model(inputs= vgg.input, outputs= model)


adam = keras.optimizers.Adam(learning_rate=0.0001)
vgg.compile(optimizer=adam, loss = 'categorical_crossentropy', metrics=['accuracy'])
vgg.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0   

In [ ]:
ES = EarlyStopping(monitor = 'val_loss',min_delta = 0.001,patience = 5,mode = 'min',restore_best_weights = True,verbose = 1)
RL = ReduceLROnPlateau(monitor = 'val_loss',factor = 0.3,patience = 5,verbose = 1,mode = 'min')
history1 = vgg.fit(datagen.flow(x_train, y_train, batch_size=20),validation_data = (x_val,y_val),epochs = 100,callbacks =[ES,RL])

Epoch 1/100


2023-01-30 17:00:22.654091: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


175/175 [==============================] - 84s 277ms/step - loss: 1.3685 - accuracy: 0.3105 - val_loss: 1.3348 - val_accuracy: 0.3352
Epoch 2/100
175/175 [==============================] - 39s 223ms/step - loss: 1.2980 - accuracy: 0.3907 - val_loss: 1.2921 - val_accuracy: 0.3490
Epoch 3/100
175/175 [==============================] - 40s 230ms/step - loss: 1.2453 - accuracy: 0.4803 - val_loss: 1.2345 - val_accuracy: 0.5270
Epoch 4/100
175/175 [==============================] - 39s 223ms/step - loss: 1.1801 - accuracy: 0.5820 - val_loss: 1.1717 - val_accuracy: 0.5741
Epoch 5/100
175/175 [==============================] - 39s 222ms/step - loss: 1.1155 - accuracy: 0.5993 - val_loss: 1.1125 - val_accuracy: 0.5798
Epoch 6/100
175/175 [==============================] - 39s 224ms/step - loss: 1.0596 - accuracy: 0.6033 - val_loss: 1.0652 - val_accuracy: 0.5867
Epoch 7/100
175/175 [==============================] - 39s 222ms/step - loss: 1.0111 - accuracy: 0.6148 - val_loss: 1.0266 - val_accurac

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



175/175 [==============================] - 43s 247ms/step - loss: 0.3824 - accuracy: 0.8647 - val_loss: 0.4418 - val_accuracy: 0.8473
Epoch 83/100
175/175 [==============================] - 41s 232ms/step - loss: 0.3776 - accuracy: 0.8679 - val_loss: 0.4414 - val_accuracy: 0.8404
Epoch 84/100
175/175 [==============================] - 41s 237ms/step - loss: 0.3791 - accuracy: 0.8656 - val_loss: 0.4364 - val_accuracy: 0.8496
Epoch 85/100
175/175 [==============================] - ETA: 0s - loss: 0.3726 - accuracy: 0.8679

In [ ]:
loss,acc = vgg.evaluate(x_test,y_test)

In [8]:
efficient=tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False,
    weights='imagenet',
    input_shape=(image_size,image_size,3),
)

2023-01-31 04:42:20.796037: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-31 04:42:20.797130: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-31 04:42:20.915969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-31 04:42:20.916943: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-31 04:42:20.917777: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

219070464/219055592 [==============================] - 1s 0us/step


In [9]:
for layer in efficient.layers:
    layer.trainable=False

In [13]:
model = efficient.output
model = GlobalAveragePooling2D()(model)
model = Dropout(0.4)(model)
model = Dense(4, activation="softmax")(model)
efficient = Model(inputs= efficient.input, outputs= model)


adam = keras.optimizers.Adam(learning_rate=0.0001)
efficient.compile(optimizer=adam, loss = 'categorical_crossentropy', metrics=['accuracy'])
efficient.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 99, 99, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 99, 99, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 99, 99, 32)   0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [14]:
ES = EarlyStopping(monitor = 'val_loss',min_delta = 0.001,patience = 5,mode = 'min',restore_best_weights = True,verbose = 1)
RL = ReduceLROnPlateau(monitor = 'val_loss',factor = 0.3,patience = 5,verbose = 1,mode = 'min')
history2 = efficient.fit(datagen.flow(x_train, y_train, batch_size=20),validation_data = (x_val,y_val),epochs = 100,callbacks =[ES,RL])

2023-01-31 04:44:40.350553: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2023-01-31 04:44:51.309337: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


175/175 [==============================] - 63s 257ms/step - loss: 1.3350 - accuracy: 0.4743 - val_loss: 0.8346 - val_accuracy: 0.6510
Epoch 2/100
175/175 [==============================] - 39s 220ms/step - loss: 0.9740 - accuracy: 0.6306 - val_loss: 0.6905 - val_accuracy: 0.7187
Epoch 3/100
175/175 [==============================] - 39s 223ms/step - loss: 0.8337 - accuracy: 0.6857 - val_loss: 0.6127 - val_accuracy: 0.7612
Epoch 4/100
175/175 [==============================] - 39s 225ms/step - loss: 0.7261 - accuracy: 0.7369 - val_loss: 0.5770 - val_accuracy: 0.7830
Epoch 5/100
175/175 [==============================] - 39s 224ms/step - loss: 0.6880 - accuracy: 0.7403 - val_loss: 0.5478 - val_accuracy: 0.7991
Epoch 6/100
175/175 [==============================] - 39s 222ms/step - loss: 0.6687 - accuracy: 0.7455 - val_loss: 0.5305 - val_accuracy: 0.8025
Epoch 7/100
175/175 [==============================] - 39s 223ms/step - loss: 0.6095 - accuracy: 0.7676 - val_loss: 0.5069 - val_accurac

In [15]:
loss,acc = efficient.evaluate(x_test,y_test)

23/23 [==============================] - 4s 158ms/step - loss: 0.4862 - accuracy: 0.8104
